In [14]:
import pandas as pd
import numpy as np
from itertools import combinations, groupby
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [15]:
aisle = pd.read_csv("C:/Users/Donna Thomas/Desktop/project files/aisles.csv")
departments = pd.read_csv("C:/Users/Donna Thomas/Desktop/project files/departments.csv")
orders = pd.read_csv("C:/Users/Donna Thomas/Desktop/project files/orders.csv")
products = pd.read_csv("C:/Users/Donna Thomas/Desktop/project files/products.csv")
order_products_prior = pd.read_csv("C:/Users/Donna Thomas/Desktop/project files/order_products__prior.csv")
order_products_train = pd.read_csv("C:/Users/Donna Thomas/Desktop/project files/order_products__train.csv")

In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
order_products_prior.tail()

,order_id,product_id,add_to_cart_order,reordered
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1
32434488,3421083,5020,10,1


In [5]:
order_products_train.tail() #training datset subset of prior products dataset. using this to make the recc model

,order_id,product_id,add_to_cart_order,reordered
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1
1384616,3421070,4724,3,1


In [6]:
orders_train = pd.merge(orders, order_products_train, on = 'order_id', how = 'inner')
orders_train_products = pd.merge(orders_train, products, on = 'product_id', how = 'inner')
orders_train_products=orders_train[orders_train['eval_set']=='train']
orders_train_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [7]:
orders = orders_train_products.set_index('order_id')['product_id'].rename('item_id')
orders.head(20)

#similar to dict. creating series with key-value pairs, finding combinations from users' each orders and products in the store

order_id
1187899      196
1187899    25133
1187899    38928
1187899    26405
1187899    39657
1187899    10258
1187899    13032
1187899    26088
1187899    27845
1187899    49235
1187899    46149
1492625    22963
1492625     7963
1492625    16589
1492625    32792
1492625    41787
1492625    22825
1492625    13640
1492625    24852
1492625    45066
Name: item_id, dtype: int64

In [8]:
#counts the number of all items and itempairs
def freq(i):
    if type(i) == pd.core.series.Series:
        return i.value_counts().rename("Freqency")
    else: 
        return pd.Series(Counter(i)).rename("Freqency")
#counter= holds the count of each item 
    
#counts the number of unique orders
def order_count(order_item):
    return len(set(order_item.index)) #set will take only unique values


def get_item_pairs(order_item):
    order_item = order_item.reset_index().values
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]      
        for item_pair in combinations(item_list, 2):
            yield item_pair                                            
            #yield: returns an object back to the function call instead of a value
            #When the function resumes, it continues execution immediately after the last yield run. 
            #This allows the code to produce a series of item-pairs over time, 
            #rather than computing them at once and sending them back like a list.

#finding support and frequency of each item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'Freqency': 'FreqencyA', 'Support': 'SupportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'Freqency': 'FreqencyB', 'Support': 'SupportB'}), left_on='item_B', right_index=True))


def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','FreqencyAB','SupportAB','FreqencyA','SupportA','FreqencyB','SupportB', 
               'ConfidenceAtoB','ConfidenceBtoA','Lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]

In [9]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    #finds freq and support
    item_stats             = freq(order_item).to_frame("Freqency")
    item_stats['Support']  = item_stats['Freqency'] / order_count(order_item) * 100

    #removes items that are below minimum support  
    qualifying_items       = item_stats[item_stats['Support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    #again removing itemsets with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    #finding support and freq of items
    item_stats             = freq(order_item).to_frame("Freqency")
    item_stats['Support']  = item_stats['Freqency'] / order_count(order_item) * 100
    item_pair_gen          = get_item_pairs(order_item)

     #finds support and freq of each itemset
    item_pairs              = freq(item_pair_gen).to_frame("FreqencyAB")
    item_pairs['SupportAB'] = item_pairs['FreqencyAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    #removing itemsets that does not satisfy minimum support
    item_pairs              = item_pairs[item_pairs['SupportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['ConfidenceAtoB'] = item_pairs['SupportAB'] / item_pairs['SupportA']
    item_pairs['ConfidenceBtoA'] = item_pairs['SupportAB'] / item_pairs['SupportB']
    item_pairs['Lift']           = item_pairs['SupportAB'] / (item_pairs['SupportA'] * item_pairs['SupportB'])
    
    
    #prints all association rules based on lift
    return item_pairs.sort_values('Lift', ascending=False)

In [10]:
rules_final= pd.DataFrame()
x=0
rules = association_rules(orders, 0.01)
item_name   = pd.read_csv("C:/Users/Donna Thomas/Desktop/project files/products.csv")
item_name1   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_pre_final = merge_item_name(rules, item_name1).sort_values('Lift', ascending=False)

Starting order_item:                1384617
Items with support >= 0.01:           11554
Remaining order_item:               1274989
Remaining orders with 2+ items:      122636
Remaining order_item:               1267339
Item pairs:                         5004378
Item pairs with support >= 0.01:      55341



In [11]:
rules_pre_final.head(20)

,itemA,itemB,FreqencyAB,SupportAB,FreqencyA,SupportA,FreqencyB,SupportB,ConfidenceAtoB,ConfidenceBtoA,Lift
0,Fancy Feast Wet Classic Chicken Feast Cat Food,Classic Tender Liver & Chicken Feast Cat Food,15,0.012231,47,0.038325,38,0.030986,0.319149,0.394737,10.299776
1,Enlighten Mint Organic,Organic Bluephoria Yerba Mate,14,0.011416,66,0.053818,31,0.025278,0.212121,0.451613,8.391515
2,Grain Free Chicken Formula Cat Food,Grain Free Turkey Formula Cat Food,14,0.011416,68,0.055449,33,0.026909,0.205882,0.424242,7.651087
3,Baby Food Pouch - Roasted Carrot Spinach & Beans,"Baby Food Pouch - Butternut Squash, Carrot & C...",15,0.012231,56,0.045664,43,0.035063,0.267857,0.348837,7.639286
4,Organic Fruit Yogurt Smoothie Mixed Berry,Apple Blueberry Fruit Yogurt Smoothie,15,0.012231,54,0.044033,47,0.038325,0.277778,0.319149,7.247991
5,Oh My Yog! Pacific Coast Strawberry Trilayer Y...,Oh My Yog! Organic Wild Quebec Blueberry Cream...,16,0.013047,56,0.045664,51,0.041586,0.285714,0.313725,6.870364
8,Baby Food Pears Squash,Stage 2 Pear Pumpkin Baby Food,17,0.013862,72,0.058710,44,0.035879,0.236111,0.386364,6.580846
9,"Apples, Pumpkin & Carrots Organic Baby Food",Stage 2 Pear Pumpkin Baby Food,17,0.013862,75,0.061157,44,0.035879,0.226667,0.386364,6.317612
47,Organic Stage 2 Pears Baby Food,Organic Stage 2 Pears & Mangos Baby Food,13,0.010600,51,0.041586,50,0.040771,0.254902,0.260000,6.252031
48,Organic Nondairy Strawberry Cashew Yogurt,Organic Cashew Nondairy Blueberry Yogurt,24,0.019570,73,0.059526,68,0.055449,0.328767,0.352941,5.929218


In [12]:
def merge_item_name(rules, item_name):
    columns = ['item_A','item_B','FreqencyAB','SupportAB','FreqencyA','SupportA','FreqencyB','SupportB', 
               'ConfidenceAtoB','ConfidenceBtoA','Lift']
    rules = (rules
                .merge(item_name, left_on='item_A', right_on='product_id')
                .merge(item_name, left_on='item_B', right_on='product_id'))
   # print(rules)
    return rules[columns]

rules_final = merge_item_name(rules, item_name).sort_values('Lift', ascending=False)
rules_final.head(20)

,item_A,item_B,FreqencyAB,SupportAB,FreqencyA,SupportA,FreqencyB,SupportB,ConfidenceAtoB,ConfidenceBtoA,Lift
0,35956,30744,15,0.012231,47,0.038325,38,0.030986,0.319149,0.394737,10.299776
1,14366,29671,14,0.011416,66,0.053818,31,0.025278,0.212121,0.451613,8.391515
2,7076,17766,14,0.011416,68,0.055449,33,0.026909,0.205882,0.424242,7.651087
3,42345,8186,15,0.012231,56,0.045664,43,0.035063,0.267857,0.348837,7.639286
4,12820,11212,15,0.012231,54,0.044033,47,0.038325,0.277778,0.319149,7.247991
5,13269,44786,16,0.013047,56,0.045664,51,0.041586,0.285714,0.313725,6.870364
8,12492,30529,17,0.013862,72,0.058710,44,0.035879,0.236111,0.386364,6.580846
9,29557,30529,17,0.013862,75,0.061157,44,0.035879,0.226667,0.386364,6.317612
47,3874,14732,13,0.010600,51,0.041586,50,0.040771,0.254902,0.260000,6.252031
48,39739,11224,24,0.019570,73,0.059526,68,0.055449,0.328767,0.352941,5.929218
